In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from datetime import datetime
from itertools import product

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=2)

<IB connected to 127.0.0.1:4004 clientId=2>

In [2]:
%%time
# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# **** Make list of qualified options stocks from the web (23 secs) *****

options_url1 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=1"
options_url2 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=2"
nse = pd.concat([pd.read_html(options_url1, header=0)[2], 
                         pd.read_html(options_url2, header=0)[2]], 
                        sort=True).drop_duplicates(subset='Symbol').reset_index(drop=True)

nse = nse.iloc[:, [1, 2, 3]]
nse.columns = ['iSymbol', 'Desc', 'Symbol']

# Note multipliers are not correct for NSE!. So get them from 5paisa!!!

paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)  # It's the second table in the url
paisa_small = paisa[['Symbol', 'Mlot']].sort_values('Symbol') # small df only with Mlot

# Check for symbols missing in nse
# pd.merge(paisa_small, nse, on='Symbol', how='left', indicator=True).query("_merge == 'left_only'" )

nse_options = pd.merge(paisa_small, nse, on="Symbol", how='left')
nse_options["Exchange"] = 'NSE'

nse_options.loc[nse_options.iSymbol.isnull(), "iSymbol"] = nse_options['Symbol'].str.slice(0,9)

# List of nse indexes (extracted from error!)
idx_list = ["BANKNIFTY", "NIFTY50", "NSEFTSE", "USDINR", "INDIAVIX", 
            "NIFTYCPSE", "NIFTYINFR", "NIFTYIT", "NIFTYMID5", "NIFTYPSE"]
nse_options = nse_options.loc[~nse_options.iSymbol.isin(idx_list), :]

# Get the Stocks qualified
stocks = [Stock(symbol=x, exchange='NSE') for x in nse_options.iSymbol]
qual_stock = ib.qualifyContracts(*stocks)

# Put the contracts to a Contract column
nse_options = nse_options.assign(Contract = pd.Series(qual_stock).values)

Wall time: 19.6 s


In [ ]:
%%time
# **** Get the close price of Underlying (14 mins) ****

def get_price(contract):
    '''Gets the price of contract
    Args:
       (contract)    = object: qualified contract object
       (exchange)    = string: ['NSE']
    
    Returns:
        price        = flost
    '''
    m_data = ib.reqMktData(contract) 
    while m_data.close != m_data.close: ib.sleep(0.01) #Wait until data is in. 
    ib.cancelMktData(contract)
    return m_data.close

list_of_prices = [get_price(c) for c in qual_stock]

nse_options["Und_Price"] = list_of_prices

In [11]:
%%time
# **** Get Expirations and Strikes. (5 secs)
contracts = [(i.symbol, '', i.secType, i.conId) for i in qual_stock]
contracts_p = [ib.reqSecDefOptParams(*p) for p in contracts]

# Filter symbol, expiration and strike from chains and make a cartesian
chains = [(i.tradingClass, i.expirations, i.strikes) for elem in contracts_p for i in elem]
L = [[[x[0]], sorted(x[1]), sorted(x[2])] for x in chains]

stock_chains = pd.DataFrame([j for i in L for j in product(*i)], 
                            columns=['Symbol', "Expiry", "Strike"])

df = pd.merge(stock_chains, nse_options, how='left', on='Symbol')
df.head()

Wall time: 4.12 s


In [ ]:
%%time
# **** Get the top 2 out-of-SD-band contracts for each symbol, expiry and right ****

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

# Populate dte for standard deviation
df['DTE'] = [get_dte(d) for d in df.Expiry]
df['DTEMax'] = df.DTE.max()
df['Exchange'] = 'NSE'

# Make iSymbol unique for contracts
sd_df = df.drop_duplicates(subset=['iSymbol', 'DTE']).reset_index(drop=True)

# function to get greatest standard deviation between dte and dte_max
def get_stdev(contract, dte, dte_max):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
       (dte_max)  = int: max days to expiry
    Returns:
       maximum standard deviation in days (int)
    '''
    sd_days = str(dte_max)+' D'
    max_bars = ib.reqHistoricalData(contract=contract, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    
    sd_dte_max = np.std([b.close for b in max_bars], ddof=1)    # standard deviation of max dte
    sd_dte = np.std([b.close for b in max_bars[:-dte]], ddof=1)  # standard deviation of dte
    
    return max(sd_dte_max, sd_dte)  # returns the greatest standard deviation

sd_df['SD'] = [get_stdev(c, d, x) for c, d, x in zip(sd_df.Contract, sd_df.DTE, sd_df.DTEMax)]

# Merge Standard Deviation to dataframe
df1 = pd.merge(df, sd_df, on=['iSymbol', 'Expiry'], how='left', suffixes=('', '_'))

df1 = df1.loc[:, ~df1.columns.str.endswith('_')].drop(['DTEMax'], axis=1)

# Designate Puts and Calls
df1['Right'] = np.where(df1.Strike < df1.Und_Price, 'P', 'C')

# Determine the Band Price
df1['Band_Price'] = np.where(df1.Right == 'P', df1.Und_Price - (df1.SD * put_sd), df1.Und_Price + (df1.SD * call_sd))

# Weed out-of-Band_Price strikes
mask = ((df1.Right == 'P') & (df1.Strike < df1.Band_Price)) | \
((df1.Right == 'C') & (df1.Strike > df1.Band_Price))

df2 = df1.loc[mask, :]  # Out-of-Band_Price

# Keep two out-of-band contracts for each right, expiry and symbol

df2["Band-Strike"] = abs(df2.Band_Price-df2.Strike) # Calculate the absolute difference
df2 = df2.sort_values(by=['Symbol', 'Expiry', 'Right', 'Band-Strike']) # Sort
df3 = df2.groupby(['Symbol', 'Expiry', 'Right']).head(2).reset_index(drop=True) # Choose the top 2

In [48]:
df3.head()

,Symbol,Expiry,Strike,Mlot,iSymbol,Desc,Exchange,Contract,Und_Price,DTE,SD,Right,Band_Price,Band-Strike
0,ACC,20180830,1800.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1541.1,24,112.133519,C,1784.439871,15.560129
1,ACC,20180830,1820.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1541.1,24,112.133519,C,1784.439871,35.560129
2,ACC,20180830,1340.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1541.1,24,112.133519,P,1344.789410,4.789410
3,ACC,20180830,1320.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1541.1,24,112.133519,P,1344.789410,24.789410
4,ACC,20180927,1780.0,400,ACC,ACC LIMITED,NSE,"Stock(conId=44652144, symbol='ACC', exchange='...",1541.1,52,107.266828,C,1773.878712,6.121288


In [ ]:
### Get the Margins

In [ ]:
### Get the Prices

In [ ]:
### Compute the RoM

In [ ]:
### Place the trades